In [ ]:
#!pip install scipy --upgrade
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy import stats


#設定星團及基本參數，需將資料檔設為星團名.csv
target_name='M45'
ra_center=56.601
dec_center=24.114
center_distance = 135.795763172189
pm=49.7443495585177
pmra_center = 19.997
pmdec_center = (-45.548)
pmra_center_error = 0.127
pmdec_center_error = 0.101
RV=5.88*(365.2422*24*60*60)/(3.085677581*10**13)
center_x = (np.cos((dec_center)/180*np.pi)*np.cos(ra_center/180*np.pi)*center_distance)
center_y = (np.cos((dec_center)/180*np.pi)*np.sin(ra_center/180*np.pi)*center_distance)
center_z = (np.sin((dec_center)/180*np.pi)*center_distance)

# 讀取星團資料
data = pd.read_csv('https://docs.google.com/spreadsheets/d/1BwgkWMv23ooESrsVrSRZ_NlFvj_yPCGRXjLpdI9X3_o/export?format=csv')
print(f"RA range {np.min(data['ra'])}  - {np.max(data['ra'])}(in degree)")
print(f"Dec range {np.min(data['dec'])} - {np.max(data['dec'])}(in degree)")
print(f"Total stars: {len(data)}")

# 計算BP-RP顏色指數、G-Rp色指數
#data['bp_rp'] = data['phot_bp_mean_mag'] - data['phot_rp_mean_mag']
#data['g_rp'] = data['phot_g_mean_mag'] - data['phot_rp_mean_mag']

# 將視差從mas轉換為arcsec，然後計算距離（parsecs）
data['distance'] = 1000 / data['parallax']
data = data[data['distance'] > 0]
data = data[data['parallax'] > 0.2]
data = data[data['phot_g_mean_mag'] < 18]  #https://doi.org/10.1051/0004-6361/201833476
# 計算G星等的絕對星等、光度、亮度
data['M_G'] = data['phot_g_mean_mag'] - (5 * np.log10(data['distance'])) + 5
data['luminosity'] = (100**0.2) ** -data['M_G']
data['brightness'] = (100**0.2) ** -data['phot_g_mean_mag']

#去除無資料星點
data = data.dropna(subset=['pmra'])
data = data.dropna(subset=['M_G'])
data = data.dropna(subset=['bp_rp'])
data = data.dropna(subset=['g_rp'])



RA range 51.73995306  - 61.39760383(in degree)
Dec range 19.72485961 - 28.55717386(in degree)
Total stars: 3214


#座標轉換

In [ ]:
#座標及單位轉換
#coords = SkyCoord(ra=data['ra'], dec=data['dec'], distance=data['distance']* u.pc, frame='icrs')
# 將角度轉換為弳度JIN流
data['Jra_rad'] = (data['ra'] - ra_center)* np.pi / 180
data['Jdec_rad'] = (data['dec'] - dec_center)* np.pi / 180
#笛卡爾座標角度轉弳度
data['ra_rad'] = data['ra']  * np.pi / 180
data['dec_rad'] = (data['dec'])* np.pi / 180

# 計算笛卡爾坐標 (x, y, z) (單位: 秒差距)
data['x'] = np.cos(data['dec_rad']) * np.cos(data['ra_rad']) *1000/ data['parallax']
data['y'] = np.cos(data['dec_rad']) * np.sin(data['ra_rad']) *1000/ data['parallax']
data['z'] = np.sin(data['dec_rad']) *1000 / data['parallax']
#-----JIN流中心畫法----------以星團中心為原點，太陽到星團中心為Z軸，直角座標散布圖
data['Jz'] = data['distance'] - center_distance
data['Jx'] = data['distance']* np.sin(data['Jra_rad'])
data['Jy'] = data['distance']* np.sin(data['Jdec_rad'])

In [ ]:
#限制徑向方向長度
Jmax = np.max(data['Jx'])
Jmin = np.min(data['Jx'])
#data = data[(data['Jz'] <=  Jmax) & (data['Jz'] >= Jmin)]
#Debug用
print(Jmin,Jmax,data['Jz'].min(),data['Jz'].max())

-12.895116597218124 12.36763625416186 -24.648915200294226 30.866639909104975


篩選成員星

##以SIMBAD自行速度和誤差篩

In [ ]:
sigma = 20
# 以error值設定倍率sigma找pmra和pmdec的上下限19.997 -45.548 [0.127 0.101 90]
pmra_min = pmra_center - sigma * pmra_center_error
pmra_max = pmra_center + sigma * pmra_center_error
pmdec_min = pmdec_center - sigma * pmdec_center_error
pmdec_max = pmdec_center + sigma * pmdec_center_error

# 選擇成員星：落在pmra和pmdec範圍內的恆星
members = data[
    (data['pmra'] >= pmra_min) & (data['pmra'] <= pmra_max) &
    (data['pmdec'] >= pmdec_min) & (data['pmdec'] <= pmdec_max)]

#僅以PM選成員星
#members = data[(data['pm']>=pm-1) & (data['pm']<=pm+1)]

##次數分配篩法

In [ ]:
"""# 定義一個函數來找到直方圖的峰值bin
n_bins=1
def find_peak_bin(data, column, n_bins=n_bins):
    n, bins = np.histogram(data[column], bins=n_bins)
    peak_index = np.argmax(n)
    peak_bin_min, peak_bin_max = bins[peak_index], bins[peak_index + 1]
    print(f"{column} peak bin: {peak_bin_min:.4f} to {peak_bin_max:.4f}")
    return peak_bin_min, peak_bin_max

# 分別找pmra和pmdec的峰值bin
pmra_min, pmra_max = find_peak_bin(data, 'pmra')
pmdec_min, pmdec_max = find_peak_bin(data, 'pmdec')

# 選擇成員星：落在pmra和pmdec峰值bin內的恆星
members = data[
    (data['pmra'] >= pmra_min) & (data['pmra'] <= pmra_max) &
    (data['pmdec'] >= pmdec_min) & (data['pmdec'] <= pmdec_max)
]
"""

'# 定義一個函數來找到直方圖的峰值bin\nn_bins=1\ndef find_peak_bin(data, column, n_bins=n_bins):\n    n, bins = np.histogram(data[column], bins=n_bins)\n    peak_index = np.argmax(n)\n    peak_bin_min, peak_bin_max = bins[peak_index], bins[peak_index + 1]\n    print(f"{column} peak bin: {peak_bin_min:.4f} to {peak_bin_max:.4f}")\n    return peak_bin_min, peak_bin_max\n\n# 分別找pmra和pmdec的峰值bin\npmra_min, pmra_max = find_peak_bin(data, \'pmra\')\npmdec_min, pmdec_max = find_peak_bin(data, \'pmdec\')\n\n# 選擇成員星：落在pmra和pmdec峰值bin內的恆星\nmembers = data[\n    (data[\'pmra\'] >= pmra_min) & (data[\'pmra\'] <= pmra_max) &\n    (data[\'pmdec\'] >= pmdec_min) & (data[\'pmdec\'] <= pmdec_max)\n]\n'

##二維高斯分布篩法

In [ ]:
"""
import matplotlib.pyplot as plt
from scipy.stats import norm
from matplotlib.patches import Ellipse

# 繪製pmra vs. pmdec散點圖
plt.figure(figsize=(10, 8))
plt.scatter(data['pmra'], data['pmdec'], alpha=0.7, edgecolor='black', linewidth=0.5)
plt.xlabel('pmra (mas/yr)')
plt.ylabel('pmdec (mas/yr)')
plt.title('Proper Motion Distribution of Stars in M67')
plt.grid(alpha=0.3)

# 計算pmra和pmdec的均值和標準差
pmra_mean, pmra_std = norm.fit(data['pmra'])
pmdec_mean, pmdec_std = norm.fit(data['pmdec'])

# 繪製1σ, 2σ, 3σ橢圓
sigmas = [1, 2, 3]
for sigma in sigmas:
    ellipse = Ellipse((pmra_mean, pmdec_mean), width=2*sigma*pmra_std, height=2*sigma*pmdec_std,
                     edgecolor='red', facecolor='none', linestyle='--', label=f'{sigma}σ')
    plt.gca().add_patch(ellipse)

plt.legend()
plt.show()

# 使用σ選擇成員星（您可以根據散點圖調整這個值）
sigma_threshold = 1
members = data[
    ((data['pmra'] - pmra_mean) / pmra_std)**2 +
    ((data['pmdec'] - pmdec_mean) / pmdec_std)**2 <= sigma_threshold**2
]
"""

"\nimport matplotlib.pyplot as plt\nfrom scipy.stats import norm\nfrom matplotlib.patches import Ellipse\n\n# 繪製pmra vs. pmdec散點圖\nplt.figure(figsize=(10, 8))\nplt.scatter(data['pmra'], data['pmdec'], alpha=0.7, edgecolor='black', linewidth=0.5)\nplt.xlabel('pmra (mas/yr)')\nplt.ylabel('pmdec (mas/yr)')\nplt.title('Proper Motion Distribution of Stars in M67')\nplt.grid(alpha=0.3)\n\n# 計算pmra和pmdec的均值和標準差\npmra_mean, pmra_std = norm.fit(data['pmra'])\npmdec_mean, pmdec_std = norm.fit(data['pmdec'])\n\n# 繪製1σ, 2σ, 3σ橢圓\nsigmas = [1, 2, 3]\nfor sigma in sigmas:\n    ellipse = Ellipse((pmra_mean, pmdec_mean), width=2*sigma*pmra_std, height=2*sigma*pmdec_std,\n                     edgecolor='red', facecolor='none', linestyle='--', label=f'{sigma}σ')\n    plt.gca().add_patch(ellipse)\n\nplt.legend()\nplt.show()\n\n# 使用σ選擇成員星（您可以根據散點圖調整這個值）\nsigma_threshold = 1\nmembers = data[\n    ((data['pmra'] - pmra_mean) / pmra_std)**2 +\n    ((data['pmdec'] - pmdec_mean) / pmdec_std)**2 <= sigma_thresh

In [ ]:
print(f"PMMax & Min RA:{pmra_max} ~ {pmra_min}, Dec:{pmdec_max} ~ {pmdec_min}")
print(f"Usable stars: {len(data)}")
print(f"Probable members: {len(members)}")

PMMax & Min RA:22.537 ~ 17.457, Dec:-43.528 ~ -47.568000000000005
Usable stars: 2387
Probable members: 1002


# 球面幾何計算自行在直角座標的向量

In [ ]:

# 轉換自行為切向速度 (秒差距/年)
v_alpha = center_distance * (1000/pmra_center) * np.cos(dec_center)
v_delta = center_distance * (1000/pmdec_center)
# 計算速度分量
v_x = -v_alpha * np.sin(ra_center) - v_delta * np.sin(dec_center) * np.cos(ra_center) + RV * np.cos(dec_center) * np.cos(ra_center)
v_y = v_alpha * np.cos(ra_center) - v_delta * np.sin(dec_center) * np.sin(ra_center) + RV * np.cos(dec_center) * np.sin(ra_center)
v_z = v_delta * np.cos(dec_center) + RV * np.sin(dec_center)

In [ ]:
#以絕對星等畫cmd，使用plotly的go
fig = go.Figure(data=go.Scatter(
    x=members['g_rp'],
    y=members['M_G'],
    mode='markers',
    marker=dict(
        color=members['g_rp'],
        colorscale='RdBu_r',
        opacity=0.7,size=10,
        line=dict(width=0.5, color='black'),
        colorbar=dict(title='G - RP'))))
fig.update_layout(
    title=f"Color-Magnitude Diagram of {target_name} Cluster Members",
    xaxis_title='G - RP',
    yaxis_title='Absolute Magnitude(G)',
    yaxis_autorange='reversed',
    width=1024,
    height=960,
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis=dict(showline=True,linewidth=1,linecolor='black',mirror=True,zeroline=True,zerolinecolor='lightgrey',zerolinewidth=1),
    yaxis=dict(showline=True,linewidth=1,linecolor='black',mirror=True,zeroline=True,zerolinecolor='lightgrey',zerolinewidth=1)
)
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='rgba(0,0,0,0.1)')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(0,0,0,0.1)')
fig.show()

#以絕對星等畫cmd，使用plotly的go，使用Bp-Rp
fig = go.Figure(data=go.Scatter(
    x=members['bp_rp'],
    y=members['M_G'],
    mode='markers',
    marker=dict(
        color=members['bp_rp'],
        colorscale='RdBu_r',
        opacity=0.7,size=10,
        line=dict(width=0.5, color='black'),
        colorbar=dict(title='Bp - RP'))))
fig.update_layout(
    title=f"Color-Magnitude Diagram of {target_name} Cluster Members",
    xaxis_title='Bp - Rp',
    yaxis_title='Absolute Magnitude(G)',
    yaxis_autorange='reversed',
    width=1024,
    height=960,
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis=dict(showline=True,linewidth=1,linecolor='black',mirror=True,zeroline=True,zerolinecolor='lightgrey',zerolinewidth=1),
    yaxis=dict(showline=True,linewidth=1,linecolor='black',mirror=True,zeroline=True,zerolinecolor='lightgrey',zerolinewidth=1)
)
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='rgba(0,0,0,0.1)')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(0,0,0,0.1)')
fig.show()

#畫空間分布圖

##直角座標成員星

In [ ]:
#繪圖、截z方向距離時需要的成員星上下界
x_min, x_max = np.min(members['x']), np.max(members['x'])
y_min, y_max = np.min(members['y']), np.max(members['y'])
z_min, z_max = np.min(members['z']), np.max(members['z'])


##直角座標場星

In [ ]:
#----用於比較的含場星資料圖
#data = data[data['phot_g_mean_mag'] < 15]
Scatter = go.Scatter3d(name='All stars scatter',
      x = data['x'],
      y = data['y'],
      z = data['z'],
#      text = 'field stars within',
      mode = 'markers',
      marker = dict(
      sizemode = 'diameter',
      opacity=1,
      line_width=0,
      sizeref = 0.1,
#      size = (data['brightness']),
      size = (data['luminosity']) ,
      sizemin=4,
      #色指數色階
      color = data['bp_rp'] ,
      colorbar_title = 'Bp-Rp',
      colorscale=[[0, 'rgb(0,0,255)'],[0.17, 'rgb(255,255,255)'],[0.3, 'rgb(255,255,0)'],[0.4, 'rgb(255,165,0)'],[0.5, 'rgb(255,0,0)'],[1, 'rgb(139,0,0)']],
      cmin=-1,
      cmax=6
#      color = data['M_G'] ,
#      colorbar_title = 'MG color',
#      colorscale=[[0, 'rgb(0,0,255)'],[0.25, 'rgb(255,255,255)'],[0.5, 'rgb(255,255,0)'],[0.6, 'rgb(255,165,0)'],[0.8, 'rgb(255,0,0)'],[1, 'rgb(139,0,0)']],
#      cmin=-5,
#      cmax=15
              )
)
Red = go.Scatter3d(name='Member Mark',
      x = members['x'],
      y = members['y'],
      z = members['z'],
#      text = 'Members position',
      mode = 'markers',
      marker = dict(
      sizemode = 'diameter',
      opacity=1,
      line=dict(width=1, color='black'),
      sizeref = 0.1,
#      size = (data['brightness']),
      size = (data['luminosity']) ,
      sizemin=4,
      color = "green")
)
VectorGC = go.Scatter3d(name='Galaxy Core',
    x=[center_x,-445.4358493],
    y=[center_y,-7113.680409],
    z=[center_z,3952.168764],
    mode='lines',
    line=dict(width=3, color='red')
)
VectorPM = go.Scatter3d(name='Proper Motion',
    x=[center_x,(center_x+v_x*100)],
    y=[center_y,(center_y+v_z*100)],
    z=[center_z,(center_z+v_z*100)],
    mode='lines',
    line=dict(width=3, color='blue')
)


Graph = [Scatter,Red,VectorGC,VectorPM]

fig = go.Figure(data=Graph)
fig.update_layout(width=1200, height=700, title = f"Star close to {target_name} Cluster",
         legend_orientation='h',legend_yanchor='bottom',
         scene = dict(aspectmode='manual',
                aspectratio=dict(x=(x_max-x_min), y=(y_max-y_min), z=(z_max-z_min)),
                xaxis=dict(title='X(pc)', titlefont_color='white',color="white", backgroundcolor="rgba(0, 0, 0, 0.5)",range=[x_min-10, x_max+10]),
                yaxis=dict(title='Y(pc)', titlefont_color='white',color="white", backgroundcolor="rgba(0, 0, 0, 0.5)",range=[y_min-10, y_max+10]),
                zaxis=dict(title='Z(pc)', titlefont_color='white',color="white",backgroundcolor="rgba(0, 0, 0, 0.3)",range=[z_min-10, z_max+10]),
                bgcolor = 'rgb(20, 24, 54)'
                 ))

fig.show()
fig.write_html(f"{target_name}.html")
#members.to_csv(f"{target_name}Menbers.csv", index=False)
#data.to_csv(f"{target_name}Data.csv", index=False)

In [ ]:
members.to_csv(f"{target_name}Menbers.csv", index=False)
data.to_csv(f"{target_name}Data.csv", index=False)

##正形成員星

In [ ]:
fig = go.Figure(data=go.Scatter3d(
    x = members['Jx'],
    y = members['Jy'],
    z = members['Jz'],
#   text = member stars,
    mode = 'markers',
    marker = dict(
        sizemode = 'diameter',
        opacity=1,
        line=dict(width=0, color='black'),
        sizeref = 0.2,
        size = (data['luminosity']) ,
        sizemin=4,
        #色指數色階
        color = data['bp_rp'] ,
        colorbar_title = 'Bp-Rp',
        colorscale=[[0, 'rgb(0,0,255)'],[0.17, 'rgb(255,255,255)'],[0.3, 'rgb(255,255,0)'],[0.4, 'rgb(255,165,0)'],[0.5, 'rgb(255,0,0)'],[1, 'rgb(139,0,0)']],
        cmin=-1,
        cmax=6
        )
))
fig.update_layout(width=1200, height=1000, title = f"{target_name} Cluster's member stars",
                  scene = dict(aspectmode='data',
                          xaxis=dict(title='X(pc)', titlefont_color='white',color="white", backgroundcolor="rgba(0, 0, 0, 0.5)"),
                          yaxis=dict(title='Y(pc)', titlefont_color='white',color="white", backgroundcolor="rgba(0, 0, 0, 0.5)"),
                          zaxis=dict(title='Distance from cluster center(pc)', titlefont_color='white',color="white",backgroundcolor="rgba(0, 0, 0, 0.3)"),
                               bgcolor = 'rgb(20, 24, 54)'
                              )
        )
fig.show()


##場星：扁形和正形

In [ ]:
#JIN流場星
fig = go.Figure(data=go.Scatter3d(
    x = data['Jx'],
    y = data['Jy'],
    z = data['distance'],
#   text = field stars,
    mode = 'markers',
    marker = dict(
        sizemode = 'diameter',
        opacity=1,
        line=dict(width=0, color='black'),
        sizeref = 0.001,
        size = (data['brightness']),
        sizemin=4,
        #色指數色階
        color = data['bp_rp'] ,
        colorbar_title = 'Bp-Rp',
        colorscale=[[0, 'rgb(0,0,255)'],[0.17, 'rgb(255,255,255)'],[0.3, 'rgb(255,255,0)'],[0.4, 'rgb(255,165,0)'],[0.5, 'rgb(255,0,0)'],[1, 'rgb(139,0,0)']],
        cmin=-1,
        cmax=6
        )
))
fig.update_layout(width=1200, height=1000, title = f"Star close to {target_name} Cluster",
                  scene = dict(aspectmode='manual',
                          aspectratio=dict(x=(max(data['Jx'])-min(data['Jx'])), y=(max(data['Jy'])-min(data['Jy'])), z=(max(data['Jz'])-min(data['Jz']))),
                          xaxis=dict(title='X(pc)', titlefont_color='white',color="white", backgroundcolor="rgba(0, 0, 0, 0.5)"),
                          yaxis=dict(title='Y(pc)', titlefont_color='white',color="white", backgroundcolor="rgba(0, 0, 0, 0.5)"),
                          zaxis=dict(title='Distance from sun(pc)', titlefont_color='white',color="white",backgroundcolor="rgba(0, 0, 0, 0.3)"),
                          bgcolor = 'rgb(20, 24, 54)'
                           )
        )
fig.show()

fig.update_layout(title = f"Star close to {target_name} Cluster(real scale)",scene_aspectmode='cube')
fig.show()

#密度分布

##密度計算

In [ ]:

def calculate_density(x, y, z, values, bins):
    """
    計算 3D 密度，允許每個維度有不同的箱子數。
    x, y, z: 坐標數組
    values: 用於計算密度的值（對於星星密度，使用 None；對於光度密度，使用光度數組）
    bins: 包含每個維度箱子數的列表或元組
    """
    if values is None:
        values = np.ones_like(x)

    # 修改此行以適應多個返回值
    statistic, edges, _ = stats.binned_statistic_dd(
        (x, y, z), values, statistic='sum', bins=bins
    )
    return statistic, edges


# 為每個維度指定不同的箱子數
bins = (30, 20, 20)  # x 維度 30 個箱子，y 維度 20 個箱子，z 維度 25 個箱子

# 計算星星密度
star_density, edges = calculate_density(data['Jx'],data['Jy'],data['Jz'], None, bins)

# 計算光度密度
luminosity_density, _ = calculate_density(data['Jx'],data['Jy'],data['Jz'], data['luminosity'], bins)

# 創建網格中心點
x_centers = (edges[0][:-1] + edges[0][1:]) / 2
y_centers = (edges[1][:-1] + edges[1][1:]) / 2
z_centers = (edges[2][:-1] + edges[2][1:]) / 2

##畫密度圖

In [ ]:

# 創建 3D 體積圖 - 星星密度
fig_stars = go.Figure(data=go.Volume(
    x=x_centers.flatten(),
    y=y_centers.flatten(),
    z=z_centers.flatten(),
    value=star_density.flatten(),
    isomin=star_density.min(),
    isomax=star_density.max(),
    opacity=0.8,  # 調整透明度
    surface_count=17,  # 調整等值面的數量
    colorscale='Viridis'  # 選擇顏色方案
))
fig_stars.update_layout(title='星星密度 3D 體積圖',xaxis_range=(-10,10),yaxis_range=(-10,10))

# 創建 3D 體積圖 - 光度密度
fig_luminosity = go.Figure(data=go.Volume(
    x=x_centers.flatten(),
    y=y_centers.flatten(),
    z=z_centers.flatten(),
    value=luminosity_density.flatten(),
    isomin=luminosity_density.min(),
    isomax=luminosity_density.max(),
    opacity=0.6,
    surface_count=17,
    colorscale='Inferno'
))
fig_luminosity.update_layout(title='光度密度 3D 體積圖')

# 顯示圖形
fig_stars.show()
fig_luminosity.show()